In [13]:
import chardet
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd

fenda = 0.1

voltagem = [4.0, 7.0, 9.0, 9.5, 10.0]
corrente = [0.45, 0.56, 0.68, 0.69, 0.70]
resistencia = 0.2

resistividade = []
temperatura = []

intensidadeLuz = []
posicaoAngular = []

refractionIndexPerVolt = []
wavelengthPerVolt = []

wavelength = []
refractionIndex = []

A = 320
B = 1
C = 0.2
D = 0.19
E = 1.635

for i in range(np.size(voltagem)):
    resistividade.append(5.65*((voltagem[i]/corrente[i]) - resistencia)/0.93)
    temperatura.append(103 + 38.1*resistividade[i] - 0.095*(resistividade[i]*resistividade[i]) + 0.000248*(resistividade[i]*resistividade[i]*resistividade[i]))
    
print(resistividade)
print(temperatura)

for volt in voltagem:
    with open(f'data/0.1mm{volt}v.txt', 'rb') as f:
            enc = chardet.detect(f.read())  # or readline if the file is large

    # print(enc['encoding'])
    file = pd.read_csv(f'data/0.1mm{volt}v.txt', header = None, sep='\s+', encoding=enc['encoding'])
    posicaoAngular.append([file[1]])
    intensidadeLuz.append([file[2]])


for array in posicaoAngular:
    for i in array:
        n = np.sqrt(((2/np.sqrt(3))*np.sin(i) + (1/2))*((2/np.sqrt(3))*np.sin(i) + (1/2)) + (3/4))
        w = A + B*(1/(n-E)) + C*(1/(n-E))*(1/(n-E)) + D*(1/(n-E))*(1/(n-E))*(1/(n-E))
        
        refractionIndexPerVolt.append(n)
        wavelengthPerVolt.append(w)
        
    refractionIndex.append(refractionIndexPerVolt)
    wavelength.append(wavelengthPerVolt)
    refractionIndexPerVolt = []
    wavelengthPerVolt = []
    

print(refractionIndex[0])
print(wavelength[0])
        
        



[52.787335722819606, 74.72580645161291, 79.19291587602784, 82.42995169082127, 85.57450076804916]
[1885.9585545795185, 2523.060031620187, 2647.6275202850907, 2736.986412485532, 2823.1160278877105]
[0     1.000000
1     1.000000
2     1.000000
3     1.000000
4     1.000000
        ...   
90    1.077426
91    1.077843
92    1.078149
93    1.078250
94    1.078449
Name: 1, Length: 95, dtype: float64]
[0     318.179149
1     318.179149
2     318.179149
3     318.179149
4     318.179149
         ...    
90    317.753745
91    317.750902
92    317.748812
93    317.748124
94    317.746760
Name: 1, Length: 95, dtype: float64]
